In [1]:
import numpy as np
import scipy.integrate as integrate
from scipy.special import legendre
import matplotlib.pyplot as plt


In [2]:
class AtTemp:
    """
    Represents a system at a specific temperature with properties relevant to eigenvalue calculations.
    
    Attributes:
        a (float): Parameter 'a' affecting the system's behavior.
        N (int): The number of points or elements considered in the system.
        omega_0 (float): A fundamental frequency of the system.
        matsubara (int): The number of Matsubara frequencies to consider.
        ng2 (float): A parameter related to the system's interaction strength.
        T (float): The temperature of the system.
    """

    def __init__(self, a, N, omega_0, matsubara, ng2):
        """
        Initializes the AtTemp instance with specified parameters and calculates initial properties.
        """
        self.a = a
        self.N = N
        self.omega_0 = omega_0
        self.matsubara = matsubara
        self.ng2 = ng2

    def omega_matsu(self):
        """
        Calculates the Matsubara frequencies for the system.
        
        Returns:
            np.ndarray: Array of Matsubara frequencies.
        """
        omega = np.asarray([(2 * (-(self.matsubara/2) + i) + 1) * np.pi * self.T for i in range(self.matsubara)]) # matsubara frequencies
        return omega
    
    def val_lambda(self):
        lambda_iso = np.asarray(
            [
                [
                    2
                    * self.omega_0
                    * self.ng2
                    / (np.square(self.omega[i] - self.omega[j]) + np.square(self.omega_0))
                    for j in range(len(self.omega))
                ]
                for i in range(len(self.omega))
            ]
        )
        #print('lambdaa', np.max(lambda_iso))
        return lambda_iso
    
    def theta_array(self):
        theta = np.linspace(0, np.pi, self.matsubara)
        return theta
    
    def Z(self):
        sigma_n = np.sum((self.omega / np.abs(self.omega)) * self.lambda_iso, axis=1)
        matrix = np.zeros((len(self.theta), len(sigma_n)))
        for i in range(len(self.theta)):
            legendre_val = legendre(self.N)(np.cos(self.theta[i]))
            for j in range(len(sigma_n)):
                matrix[i, j] = 1 + (np.pi*self.T/(self.omega[j])) * (1 + self.a * legendre_val) * sigma_n[j]
        return matrix

    def phi_integration_simpson(self):
        Zz = self.zz
        integration_list = []
        for i in range(Zz.shape[1]):
            denominator = Zz[:, i]
            numerator = (1 + self.a * (legendre(self.N)(np.cos(self.theta))) )**2 
            sub_integrand = np.divide(numerator, denominator) # for this step we know it will work when len(thera) = len(omega). But what when it doesn't?
            integrand = np.sin(self.theta) * sub_integrand
            integrated_element = integrate.simpson(integrand, self.theta)
            integration_list.append(integrated_element)
        integration = np.array(integration_list)
        return integration
        
    def cal_matrix(self):
        k_matrix = (np.pi/2) * self.T * self.lambda_iso * self.integration
        matrix = np.divide(k_matrix, np.abs(self.omega))
        return matrix
    
    def eigenvalue_prob(self):
        eigenvalues, _ = np.linalg.eig(self.matrix)
        return eigenvalues
    
    def max_eigenvalue(self):
        return np.max(self.eigenvalues)
    
    def t_iter_update(self, T_new):
        self.T = T_new
        self.omega = self.omega_matsu()
        self.lambda_iso = self.val_lambda()
        self.theta = self.theta_array()
        self.zz = self.Z()
        self.integration = self.phi_integration_simpson()
        self.matrix = self.cal_matrix()
        self.eigenvalues = self.eigenvalue_prob()
        updated_eigenvalue = self.max_eigenvalue()
        return updated_eigenvalue

In [3]:
class ForAvalue(AtTemp):
    """
    Extends AtTemp to iteratively calculate and evaluate eigenvalues over a range of temperatures 
    until a specified eigenvalue limit is exceeded.
    
    Attributes:
        eignevalue_limit (float): The target eigenvalue limit for iteration.
        T_start (float): The starting temperature for the iteration.
        T_steps (float): The temperature increment for each iteration step.
    """
    def __init__(self, a, N, omega_0, matsubara, ng2, T_low, T_high, tol):
        """
        Initializes the ForAvalue instance with specific system parameters and iteration settings.
        """
        super().__init__(a, N, omega_0, matsubara, ng2) # initialize the parent class
        self.T_low = T_low
        self.T_high = T_high
        self.tol = tol

    def iter_over_t(self):
        T_low = self.T_low
        T_high = self.T_high
        tol = self.tol
        eigenvalue_low = self.t_iter_update(T_new=T_low)
        eigenvalue_high = self.t_iter_update(T_new=T_high)
        T_values = [T_low, T_high]
        eigenvalues_forT = [eigenvalue_low, eigenvalue_high]
        # Make sure that the interval brackets the target eigenvalue
        if not (eigenvalue_low - 1) * (eigenvalue_high - 1) < 0:
            print("Initial T values do not bracket the target eigenvalue of 1.")
            return None
        max_iterations = 10000  # Limit the number of iterations to prevent infinite loops
        iteration = 0  # Initialize iteration counter
        while iteration < max_iterations:
            T_mid = (T_low + T_high) / 2
            eigenvalue_mid = self.t_iter_update(T_new=T_mid)
            T_values.append(T_mid)
            eigenvalues_forT.append(eigenvalue_mid)
            if abs(eigenvalue_mid - 1) < tol:
                print(f"Converged: T = {T_mid}, eigenvalue = {eigenvalue_mid}")
                return T_values, eigenvalues_forT

            if (eigenvalue_mid - 1) * (eigenvalue_low - 1) < 0:
                T_high = T_mid
                eigenvalue_high = eigenvalue_mid
            else:
                T_low = T_mid
                eigenvalue_low = eigenvalue_mid
            iteration += 1  # Increment the iteration counter
        print("Reached maximum iterations without converging.")
        return None  # This line can be omitted, as Python implicitly returns None

In [8]:
class ExploreEigenvalue:
    def __init__(self, a_values, N, omega_0, matsubara, ng2, T_low, T_high, tol):
        self.a_values = a_values
        self.N = N
        self.omega_0 = omega_0
        self.matsubara = matsubara
        self.ng2 = ng2
        self.T_low = T_low
        self.T_high = T_high
        self.tol = tol
        self.data = self.iter_over_a()
        
    def iter_over_a(self):
        parent_dict = {}
        for a in self.a_values:
            a_value_data = ForAvalue(a, self.N, self.omega_0, self.matsubara, self.ng2, self.T_low, self.T_high, self.tol)
            T_values, eigenvalues_forT = a_value_data.iter_over_t()
            nested_dict = {}
            nested_dict['Temperature'] = T_values
            nested_dict['eigenvalues'] = eigenvalues_forT
            parent_dict[a] = nested_dict
            
        return parent_dict

In [9]:
omega_0 = 0.05 # phonon frequency
ng2 = 0.014 # coupling strength?

#T = 0.00014
matsubara = 100
N = 2

T_low = 0.0023
T_high = 0.01
tol = 1e-4

In [10]:
#a = [-0.3, -0.2, -0.1, -0.05, -0.01, 0.0, 0.01, 0.05, 0.1, 0.2, 0.3]
a=[0.0]


In [11]:
lessdo = ExploreEigenvalue(a, N, omega_0, matsubara, ng2, T_low, T_high, tol)

Converged: T = 0.00248046875, eigenvalue = 0.9999097778953348


In [12]:
parent_dict = lessdo.data

In [13]:
parent_dict

{0.0: {'Temperature': [0.0023,
   0.01,
   0.00615,
   0.004225,
   0.0032624999999999998,
   0.00278125,
   0.002540625,
   0.0024203124999999997,
   0.00248046875],
  'eigenvalues': [1.0264376804752264,
   0.5413794180743747,
   0.6878317881072744,
   0.8148156490782177,
   0.9042408944040613,
   0.9598505057598274,
   0.9915097396088499,
   1.0085235153146086,
   0.9999097778953348]}}